(1) Import required packages

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm 
import matplotlib.pyplot as plt

(2) Provide the path here at which find input data and save the output

In [4]:
direc = '/Users/snagel/Dropbox/hdrive/teaching/Booth/AdvancedInvestments/2024/output/'

---------------------------------------------------------------------------------------
Part 1
---------------------------------------------------------------------------------------

(3) Read data and calculate returns 

In [3]:
# Read data from Excel file
M = pd.read_excel('PS2data1.xlsx')

# Extract specific columns from the dataframe
ret = M[['rvwind', 'rglobalstocks', 'rvalue', 'rsmall', 'rtreas', 'rglobalbonds', 'rcorp', 'rreit', 'rgold']].values
rf = M['rf'].values
month = M['month'].values
T, N = ret.shape

# Calculate excess returns
excret = ret - np.tile(rf.reshape(-1, 1), (1, N))

(4) In-sample plug-in approach

In [ ]:
m = np.mean(excret, axis=0)
S = np.cov(excret, rowvar=False, ddof=1)
w = np.linalg.inv(S) @ m
wr = w / np.sum(w)
pr = excret @ wr

print('In-sample plug-in')
print(wr)
psr = np.sqrt(12) * np.mean(pr) / np.std(pr, ddof=1)
print(psr) 

(5) Out-of-sample analysis for various degrees of shrinkage (including no shrinkage) 

In [ ]:
win = 239  # (default = 239) 
pr = np.empty((T - win - 1, 101))
wts = np.empty((T - win, 101, N))

for j in range(101): 
    phi = j / 100 

    for i in range(T - win - 1): 
        x = excret[i:i + win + 1, :]    # selects elements i to i+win 
        m = np.mean(x, axis=0)
        S = np.cov(x, rowvar=False, ddof=1)
        sd = np.sqrt(np.diag(S))
        w = np.linalg.inv(S) @ m
        wr = w / np.sum(w)
        wr = wr * phi + (1 - phi) * (1 / N)
        r = excret[i + win + 1, :]
        pr[i, j] = wr @ r
        wts[i, j, :] = wr

psr = np.sqrt(12) * np.mean(pr, axis=0) / np.std(pr, axis=0, ddof=1)

# pick out the results for the no-shrinkage case 
print('OOS, no shrinkage SR')
print(psr[100])

# Plotting the weights over time 
plt.figure()
plt.plot(wts[:, 100, :], linewidth=2)
plt.legend(['Stocks', 'I-Stocks', 'Value', 'Small', 'T-Bonds', 'I-Bonds', 'C-Bonds', 'REITS', 'Gold'],
           loc='upper right', fontsize=8)
plt.xlabel('Time')
plt.ylabel('Weights')
plt.savefig(direc + 'OOSwts.pdf', format='pdf')
plt.show()

(6) Optimal shrinkage 

In [ ]:
# Looking for optimal degree of shrinkage
ind = np.argmax(psr)
maxwt = wts[:, ind, :]
phis = np.arange(101) / 100
print('Opt. shrinkage parameter value:', phis[ind])  
print('Opt. shrinkage OOS SR:', psr[ind])
plt.figure()
plt.plot(phis, psr)
plt.xlabel('\u03D5')
plt.ylabel('Sharpe Ratio')
plt.savefig(direc + 'OOSshrinkSR.pdf', format='pdf')
plt.show()

(7) Plot weights implied by optimal shrinkage

In [ ]:
plt.figure()
plt.plot(maxwt, linewidth=2)
plt.legend(['Stocks', 'I-Stocks', 'Value', 'Small', 'T-Bonds', 'I-Bonds', 'C-Bonds', 'REITS', 'Gold'],
           loc='upper right', fontsize=14)
plt.xlabel('Time')
plt.ylabel('Weights')
plt.savefig(direc + 'OOSmaxwt.pdf', format='pdf')
plt.show()


----------------------------------------------------------------------------------------
Part 2
----------------------------------------------------------------------------------------

(8) Read data for returns and predictor variables

In [ ]:
# Read data from Excel file
M = pd.read_excel('PS2data2.xlsx')

# Extract specific columns from the dataframe
ret = M['rvwind'].values
rf = M['rf'].values
dy = M['repadjdy'].values

# Form lagged log P/D and lagged returns
lpd = -np.log(dy[:-1])
lr = ret[:-1] - rf[:-1]

# Excess returns
xr = ret[1:] - rf[1:]
T = len(xr)

(9) In-sample analysis 

In [ ]:
X_lr = sm.add_constant(lr) 
model = sm.OLS(xr,X_lr)
results = model.fit() 
print(results.summary()) 

X_lpd = sm.add_constant(lpd) 
model = sm.OLS(xr,X_lpd)
results = model.fit() 
print(results.summary()) 

(10) Out-of-sample analysis for various degrees of shrinkage (including no shrinkage)

In [ ]:
win = 239  # (default = 239) 
pr = np.empty((T - win - 1, 101))
wts = np.empty((T - win - 1, 101))

for j in range(101):
    phi = j / 100

    for i in range(T - win - 1):
        y = xr[i:i + win + 1]      # selects elements i to i+win 
        x = lr[i:i + win + 1]
        X = sm.add_constant(x) 
        model = sm.OLS(y,X)
        results = model.fit() 
        b = results.params 
        w = np.array([1, lr[i + win + 1]]) @ b
        pr[i, j] = xr[i + win + 1] * (w * phi + (1 - phi) * 0.01)
        wts[i, j] = w

srs = np.sqrt(12) * np.mean(pr, axis=0) / np.std(pr, axis=0, ddof=1)
print('constant weight strategy SR:', srs[0])
print('no shrinkage strategy SR:', srs[-1])


(11) Optimal shrinkage 

In [ ]:
ind = np.argmax(srs)
maxwt = wts[:, ind]
print('OOS, opt. shrinkage parameter value:', phis[ind]) 
print('Opt. shrinkage OOS SR:', srs[ind])
     
phis = np.arange(101) / 100
f = plt.figure()
plt.plot(phis, srs)
plt.ylabel('Sharpe ratio')
plt.xlabel('\u03D5')
plt.savefig(direc + 'OOSpredSR.pdf', format='pdf')
plt.show()



(12) Plot weights implied by optimal shrinkage 

In [ ]:
plt.figure()
plt.plot(maxwt, linewidth=2)
plt.xlabel('Time')
plt.ylabel('Weights')
plt.savefig(direc + 'OOSmaxwt2.pdf', format='pdf')
plt.show()